### **Load images from directory**
- Use flow_from_dataframe function to not have to save images in subfolders according to their label.
- Build pandas dataframe containing the filenames and the according labels

In [44]:
import pandas as pd
import os
import tensorflow as tf

2022-12-02 13:16:31.983167: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### **Useful functions**

In [37]:
def get_label(string):
    string  = ' '.join(string.split('/')[-1].replace('.png', '').split(' ')[0:-1])
    return string

#### **Load file names into a list and create pandas dataframe**
Need to remove .gitkeep file from list

In [38]:
path = '../data/raw/'
filenames = os.listdir(path)
filenames.remove('.gitkeep')

Build pandas dataframe

In [34]:
df = pd.DataFrame(filenames, columns=['Filenames'])

#### **Get labels from filenames**

In [54]:
# Retrieve the label from the path of the pictures
df['Label'] = df['Filenames'].apply(lambda x: get_label(x))

Check whether number of images per category is right

In [39]:
df['Label'].value_counts()

3023 Plate 1x2                         800
4286 roof tile 1x3                     800
3005 brick 1x1                         800
2357 brick corner 1x2x2                800
54200 roof tile 1x1                    800
6143 Brick D16 w cross                 800
3004 brick 1x2                         800
3020 plate 2x4                         800
3024 plate 1x1                         800
3068 flat tile 2x2                     800
3659 brick bow 1x4                     800
3022 Plate 2x2                         800
3046 roof corner inside tile 2x2       800
43093 Bush 2M friction - Cross axle    800
99301 roof tile inside 3x3             800
33909 plate 2 knobs 2x2                800
4490 brick bow 1x3                     800
3623 Plate 1x3                         800
3700 Technic Brick 1x2                 800
3062 Round Brick 1x1                   800
3021 plate 2x3                         800
3675 roof tile outside 3x3             800
3001 brick 2x4                         800
15672 roof 

Instantiate ImageDataGenerator and then use flow_from_dataframe to get training and validation data

In [55]:
val_split = 0.1
image_size = (150, 150)

datagen = tf.keras.preprocessing.image.ImageDataGenerator(validation_split = val_split)
train_generator = datagen.flow_from_dataframe(df,
                                              directory=path,
                                              x_col='Filenames',
                                              y_col='Label',
                                              target_size=image_size,
                                              color_mode='grayscale',
                                              class_mode='categorical',
                                              subset='training')
validation_generator = datagen.flow_from_dataframe(df,
                                              directory=path,
                                              x_col='Filenames',
                                              y_col='Label',
                                              target_size=image_size,
                                              color_mode='grayscale',
                                              class_mode='categorical',
                                              subset='validation')

Found 36000 validated image filenames belonging to 50 classes.
Found 4000 validated image filenames belonging to 50 classes.
